In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.datasets import fetch_openml
%matplotlib inline
plt.style.use('bmh')

In [2]:
class kNNClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
        self.data = np.empty((1,1))
        self.labels = np.empty((1,1))
    
    def fit(self, X, y):
        self.data = X
        self.labels= y
        
    def _predict_one_point(self,point):
        dist = np.linalg.norm(self.data-point,axis=1)
        k_smallets = np.argpartition(dist, self.n_neighbors)[:self.n_neighbors]
        label_count = np.unique(self.labels[k_smallets],return_counts=True)
        return label_count[0][label_count[1].argmax()]
        
    
    def predict(self, X):
        preds = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            preds[i]=self._predict_one_point(X[i])
        return preds.T[0]
    
    def score(self, predictions, true_labels):
        
        return (np.count_nonzero(predictions-true_labels.astype("int"))/predictions.size)
        
        